In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.d_sale_price_store_sku_daily
(
    calday DATE,
    store_id STRING,
    product_id STRING,
    sale_price DECIMAL(38,9),
    sale_unit STRING,
    app STRING,
    cond_type STRING,
    cond_no STRING,
    sale_org STRING,
    distribution_channel STRING,
    valid_from DATE,
    valid_to DATE,
    sale_price_type STRING
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")


In [0]:
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW temp_d_sale_price_store_sku_daily AS
WITH
STORE_ATTR AS (
  SELECT a.*
  FROM
    {catalog_name}.udp_wcm_gold_vcm_dwh.d_sale_price_store_attr a
    INNER JOIN (
      SELECT
        store_id,
        CASE
          WHEN business_unit = '1500' THEN '20'
          WHEN business_unit = '2000' THEN '10'
        END distribution_channel
      FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store
      WHERE business_unit IN ('1500', '2000')
    ) b ON a.store_id = b.store_id AND a.distribution_channel = b.distribution_channel
),
SALE_PRICE_SITE AS (
  SELECT
    a.app,
    a.cond_type,
    a.cond_no,
    a.sale_org,
    a.distribution_channel,
    a.store_id,
    a.product_id,
    a.sale_unit,
    a.valid_from,
    a.valid_to,
    'SITE' sale_price_type,
    1 priority
  FROM
    {catalog_name}.udp_wcm_gold_vcm_dwh.d_sale_price_site a
    INNER JOIN (
      SELECT
        store_id,
        CASE
          WHEN business_unit = '1500' THEN '20'
          WHEN business_unit = '2000' THEN '10'
        END distribution_channel
      FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store
      WHERE business_unit IN ('1500', '2000')
    ) b ON a.store_id = b.store_id AND a.distribution_channel = b.distribution_channel
),
SALE_PRICE_GROUP AS (
  SELECT
    b.app,
    b.cond_type,
    b.cond_no,
    b.sale_org,
    b.distribution_channel,
    a.store_id,
    b.product_id,
    b.sale_unit,
    b.valid_from,
    b.valid_to,
    'GROUP' sale_price_type,
    2 priority
  FROM
    STORE_ATTR a
    INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_sale_price_group b
      ON a.sale_org = b.sale_org AND a.distribution_channel = b.distribution_channel AND a.price_group = b.price_group
  WHERE
    EXTRACT(YEAR FROM b.valid_to ) <> 7777
    AND EXTRACT(YEAR FROM b.valid_from ) <> 7777
    AND (DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) BETWEEN valid_from AND valid_to)
),
SALE_PRICE_LIST AS (
  SELECT
    b.app,
    b.cond_type,
    b.cond_no,
    b.sale_org,
    b.distribution_channel,
    a.store_id,
    b.product_id,
    b.sale_unit,
    b.valid_from,
    b.valid_to,
    'LIST' sale_price_type,
    3 priority
  FROM
    STORE_ATTR a
    INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_sale_price_list b
      ON a.sale_org = b.sale_org AND a.distribution_channel = b.distribution_channel AND a.price_list = b.price_list
  WHERE
    EXTRACT(YEAR FROM b.valid_to ) <> 7777  
    AND EXTRACT(YEAR FROM b.valid_from ) <> 7777
    AND (DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) BETWEEN valid_from AND valid_to)
),
SALE_PRICE_CHANNEL AS (
  SELECT
    b.app,
    b.cond_type,
    b.cond_no,
    b.sale_org,
    b.distribution_channel,
    a.store_id,
    b.product_id,
    b.sale_unit,
    b.valid_from,
    b.valid_to,
    'CHANNEL' sale_price_type,
    4 priority
  FROM
    STORE_ATTR a
    INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_sale_price_channel b
      ON a.sale_org = b.sale_org AND a.distribution_channel = b.distribution_channel
  WHERE
    EXTRACT(YEAR FROM b.valid_to ) <> 7777
    AND EXTRACT(YEAR FROM b.valid_from ) <> 7777
    AND (DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) BETWEEN valid_from AND valid_to)
),
SALE_PRICE_COMBINE AS (
  SELECT *
  FROM (
    SELECT * FROM SALE_PRICE_SITE
    UNION ALL
    SELECT * FROM SALE_PRICE_GROUP
    UNION ALL
    SELECT * FROM SALE_PRICE_LIST
    UNION ALL
    SELECT * FROM SALE_PRICE_CHANNEL
  )
  QUALIFY ROW_NUMBER() OVER (PARTITION BY app, cond_type, sale_org, distribution_channel, store_id, product_id, sale_unit ORDER BY priority) = 1
)

SELECT
  DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)  calday,
  b.store_id,
  b.product_id,
  a.rate * 100 sale_price,
  b.sale_unit,
  b.app,
  b.cond_type,
  b.cond_no,
  b.sale_org,
  b.distribution_channel,
  b.valid_from,
  b.valid_to,
  b.sale_price_type
FROM
{catalog_name}.udp_wcm_gold_vcm_dwh.d_sale_price_daily a
  INNER JOIN SALE_PRICE_COMBINE b ON a.app = b.app AND a.cond_type = b.cond_type AND a.cond_rec_num = b.cond_no
  INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product c ON b.product_id = c.product_id
WHERE
  (a.del_indicator IS NULL OR a.del_indicator <> 'X')
  AND a.rate IS NOT NULL
""")

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.d_sale_price_store_sku_daily
SELECT * FROM temp_d_sale_price_store_sku_daily""")

In [0]:
spark.sql("DROP VIEW IF EXISTS temp_d_sale_price_store_sku_daily")